[View in Colaboratory](https://colab.research.google.com/github/vochicong/ai-math/blob/master/TensorFlow_Hub.ipynb)

# [TensorFlow (TF) Hub](https://www.tensorflow.org/hub/installation)

In [2]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

    100% |████████████████████████████████| 48.0MB 28kB/s 
    100% |████████████████████████████████| 3.1MB 423kB/s 
  Found existing installation: tensorboard 1.6.0
    Uninstalling tensorboard-1.6.0:
      Successfully uninstalled tensorboard-1.6.0
  Found existing installation: tensorflow 1.6.0
    Uninstalling tensorflow-1.6.0:
      Successfully uninstalled tensorflow-1.6.0
    100% |████████████████████████████████| 61kB 1.9MB/s 


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [4]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

  return train_df, test_df

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

84131840/84125825 [==============================] - 4s 0us/step


,sentence,sentiment,polarity
0,If you want to see a movie that terribly mixes...,1,0
1,"Oh, come on people give this film a break. The...",1,0
2,A female vampire kills young women and paints ...,2,0
3,"As a fan of the Sookie Stackhouse books, I fin...",1,0
4,This film show peoples in the middle of the ho...,8,1


In [0]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, test_df["polarity"], shuffle=False)

In [0]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")